In [21]:
import os

os.chdir("/master/nplatt/sch_man_nwinvasion")

## Get random samples
Randomly select 10 individuals from each population. This allows us see if/how sample size influences our results.

In [2]:
%%bash 

mkdir results/n10

#get lists of each population and random subset (of 10)
for POP in brazil tanzania senegal niger; do
    cat results/lists/$POP.list | shuf | shuf | shuf | shuf | head -n 10 >results/n10/"$POP"_n10.list
done

cat results/n10/*_n10.list >results/n10/n10.list

vcftools \
    --vcf results/variant_filtration/smv7_ex_autosomes.vcf \
    --keep results/n10/n10.list \
    --recode \
    --recode-INFO-all \
    --stdout \
    >results/n10/smv7_ex_autosomes_n10.vcf

mkdir: cannot create directory `results/n10': File exists

VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--vcf results/variant_filtration/smv7_ex_autosomes.vcf
	--keep results/n10/n10.list
	--recode-INFO-all
	--recode
	--stdout

Keeping individuals in 'keep' list
After filtering, kept 40 out of 156 Individuals
Outputting VCF file...
After filtering, kept 475081 out of a possible 475081 Sites
Run Time = 198.00 seconds


## MSPRIME
Simulate neutral VCFS for each population from 10 individuals

In [24]:
%%bash

mkdir -p results/n10/msprime/logs

CONDA="conda activate sch_man_nwinvasion-msprime;"
QSUB="qsub -V -cwd -S /bin/bash -q all.q -j y -pe smp 1 "    

for POP in tanzania senegal niger brazil; do
    mkdir -p results/n10/msprime/$POP
    for I in $(seq 1 342); do
        MSPRIME_CMD="python code/msprime-qsub_n10.py $POP $I"       
        echo "$CONDA $MSPRIME_CMD" | $QSUB -N $POP"_"$I -o results/n10/msprime/logs/$POP"_"$I.log
    done
done

Your job 5665356 ("tanzania_1") has been submitted
Your job 5665357 ("tanzania_2") has been submitted
Your job 5665358 ("tanzania_3") has been submitted
Your job 5665359 ("tanzania_4") has been submitted
Your job 5665360 ("tanzania_5") has been submitted
Your job 5665361 ("tanzania_6") has been submitted
Your job 5665362 ("tanzania_7") has been submitted
Your job 5665363 ("tanzania_8") has been submitted
Your job 5665364 ("tanzania_9") has been submitted
Your job 5665365 ("tanzania_10") has been submitted
Your job 5665366 ("tanzania_11") has been submitted
Your job 5665367 ("tanzania_12") has been submitted
Your job 5665368 ("tanzania_13") has been submitted
Your job 5665369 ("tanzania_14") has been submitted
Your job 5665370 ("tanzania_15") has been submitted
Your job 5665371 ("tanzania_16") has been submitted
Your job 5665372 ("tanzania_17") has been submitted
Your job 5665373 ("tanzania_18") has been submitted
Your job 5665374 ("tanzania_19") has been submitted
Your job 5665375 ("ta

In [ ]:
#now get the vcfs
#python code/msprime-probe_snps_from_vcf.py (modified for n10)

import msprime
import os
import subprocess
import glob


#now create bed for the new "sim" chr1
with open('data/renamed-sma_agilent_baits.v7.0.chr_reorderd.bed', 'r') as in_bed:
    with open('results/msprime/sim_probes.bed', 'w') as out_bed:
        for bed_entry in in_bed:
            chrom, start, stop = bed_entry.rstrip().split("\t")
            if chrom == "SM_V7_1":
                out_bed.write("1\t{}\t{}\n".format(start, stop))

bed = 'results/msprime/sim_probes.bed'
#now loop through all of the sim vcf files to get snps at probed regions
for pop in ["niger", "senegal", "tanzania", "brazil"]:
    out_dir = "results/n10/msprime/{}".format(pop)
    
    sim_vcfs = glob.glob("{}/chr1_*_rep_*.vcf".format(out_dir))
    for sim_vcf in sim_vcfs:

        probed_vcf = sim_vcf.replace(".vcf", "_probed.vcf")         
        jid = "probe_{}".format(probed_vcf.split("/")[-1])
        log = "{}/logs".format(out_dir)

        vcf_cmd = "vcftools --vcf {} --bed {} --recode --recode-INFO-all --stdout >{}".format(sim_vcf, bed, probed_vcf)
        qsub_cmd =  "qsub -V -cwd -S /bin/bash -q all.q -j y -pe smp 3 -N {} -o {}".format(jid, log)
        conda_cmd = "conda activate sch_man_nwinvasion-msprime"

        cmd ="echo \"{}; {}\" | {}".format(conda_cmd, vcf_cmd, qsub_cmd)

        #run vcf cmd
        #process = subprocess.Popen(cmd.split(""),
        #                     stdout=subprocess.PIPE, 
        #                     stderr=subprocess.PIPE)
        !{cmd}
        

## HSCAN

HSCAN identifies regions under selection based on the length of haplotypes.  (citation).  We used a custom code to convert HSCAN to VCF input formats.

In [22]:
%%bash

cd ~/sch_man_nwinvasion/

CONDA="conda activate sch_man_nwinvasion-hscan"
QSUB="qsub -V -cwd -S /bin/bash -q all.q -j y -pe smp 1 "    
OUTDIR="results/n10/hscan"

if [ ! -d $OUTDIR ] 
then
    mkdir $OUTDIR
fi

mkdir -p $OUTDIR/logs

#REAL DATA
for POP in brazil tanzania niger senegal; do
    mkdir $OUTDIR/$POP
        for i in $(seq 1 7 ); do
            CHROM="SM_V7_"$i
    
            #get chrom vcf
            VCF_CMD="vcftools \
                --vcf results/n10/smv7_ex_autosomes_n10.vcf \
                --keep results/n10/"$POP"_n10.list \
                --chr $CHROM \
                --recode \
                --recode-INFO-all \
                --stdout \
                >$OUTDIR/$POP/"$CHROM"_"$POP".vcf"

            #convert vcf to hscan
            CONV_CMD="python code/vcf2hscan.py \
                $OUTDIR/$POP/"$CHROM"_"$POP".vcf \
                $OUTDIR/$POP/"$CHROM"_"$POP".hscan-in"

            #convert to hscan
            HSCAN_CMD="bin/H-scan \
                -i $OUTDIR/$POP/"$CHROM"_"$POP".hscan-in \
                -g 10000 \
                >$OUTDIR/$POP/"$CHROM"_"$POP".hscan-out"

        echo "$CONDA; $VCF_CMD; $CONV_CMD; $HSCAN_CMD" | $QSUB -N $POP"_"$CHROM -o $OUTDIR/logs/$POP"_"$CHROM.log

    done
done



Your job 5666864 ("brazil_SM_V7_1") has been submitted
Your job 5666865 ("brazil_SM_V7_2") has been submitted
Your job 5666866 ("brazil_SM_V7_3") has been submitted
Your job 5666867 ("brazil_SM_V7_4") has been submitted
Your job 5666868 ("brazil_SM_V7_5") has been submitted
Your job 5666869 ("brazil_SM_V7_6") has been submitted
Your job 5666870 ("brazil_SM_V7_7") has been submitted
Your job 5666871 ("tanzania_SM_V7_1") has been submitted
Your job 5666872 ("tanzania_SM_V7_2") has been submitted
Your job 5666873 ("tanzania_SM_V7_3") has been submitted
Your job 5666874 ("tanzania_SM_V7_4") has been submitted
Your job 5666875 ("tanzania_SM_V7_5") has been submitted
Your job 5666876 ("tanzania_SM_V7_6") has been submitted
Your job 5666877 ("tanzania_SM_V7_7") has been submitted
Your job 5666878 ("niger_SM_V7_1") has been submitted
Your job 5666879 ("niger_SM_V7_2") has been submitted
Your job 5666880 ("niger_SM_V7_3") has been submitted
Your job 5666881 ("niger_SM_V7_4") has been submitted


### Calculate P value from Hscan and Plot

In [17]:
#when these are done running. plot

################################################################################
#make table and plot 

#calculate p-values
import scipy.stats
import statsmodels.api as sm
import statsmodels as sm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob

In [18]:
#make sure that all stops are not gt chrom length
chr_length = {}
#genome_size = 0
with open('/master/nplatt/sch_man_nwinvasion/data/genomes/Smansoni_v7.fa.fai', 'r') as fai:
    for entry in fai:
        chrom, length, *offset = entry.rstrip().split("\t")
        chr_length[chrom]=int(length)

    cumul_start={}
    cumul_start['SM_V7_1']=0
    cumul_start['SM_V7_2']= cumul_start['SM_V7_1'] + chr_length['SM_V7_1']
    cumul_start['SM_V7_3']= cumul_start['SM_V7_2'] + chr_length['SM_V7_2']
    cumul_start['SM_V7_4']= cumul_start['SM_V7_3'] + chr_length['SM_V7_3']
    cumul_start['SM_V7_5']= cumul_start['SM_V7_4'] + chr_length['SM_V7_4']
    cumul_start['SM_V7_6']= cumul_start['SM_V7_5'] + chr_length['SM_V7_5']
    cumul_start['SM_V7_7']= cumul_start['SM_V7_6'] + chr_length['SM_V7_6']
    scanned_size = cumul_start['SM_V7_7'] + chr_length['SM_V7_7']

In [21]:
%%bash

head results/n10/hscan/niger/SM_V7_1_niger.hscan-out

x	H
302288	0
705190	127.556
705210	157.333
705322	137.556
705337	134.756
705340	146.356
705343	145.156
705389	145.156
705395	134.756


In [13]:
"results/n10/hscan/{}_n10/{}_{}_n10.hscan-out".format(pop, chrom, pop)

'results/n10/hscan/niger_n10/SM_V7_1_niger_n10.hscan-out'

In [22]:
for pop in ["niger",     "senegal",     "brazil",     "tanzania"]:
    print(pop)
    chrom_s=[]
    pos_s=[]
    h_s=[]

    for i in range(1,8):
        chrom = "SM_V7_{}".format(i)
        
        with open("results/n10/hscan/{}/{}_{}.hscan-out".format(pop, chrom, pop), 'r') as hscan_file:
            next(hscan_file)
            for calc in hscan_file:
                
                pos, h = calc.split("\t")
                
                chrom_s.append(chrom)
                h_s.append(h)
                pos_s.append(pos)

    chrom_s=np.array(chrom_s)
    pos_s=np.array(pos_s)
    h_s=np.array(h_s).astype(np.float)
    
    ##calculate qs    
    #z_s=scipy.stats.zscore(h_s)
    #p_s=scipy.special.ndtr(-z_s)     
    #reject_s, q_s, sidak, bonf =sm.stats.multitest.multipletests(p_s, alpha=0.01, method="fdr_bh")
    #logq_s=-np.log10(q_s)

    #calculate max neutral h score
    h_max = 0.0
    sim_files = glob.glob("results/hscan/{}-msprime/*hscan-out".format(pop))     
    for sim_file in sim_files:
        with open(sim_file, 'r') as in_sim_file:
            next(in_sim_file)
            for sim_entry in in_sim_file:
                 x, h = sim_entry.rstrip().split("\t")
                 if (h != "H") and (float(h)>float(h_max)):
                    h_max=float(h)

    #calculate qs (with simulated hmax
    z_s=scipy.stats.zscore(np.insert(h_s, 0, h_max))
    p_s=scipy.special.ndtr(-z_s)   
    reject_s, q_s, sidak, bonf =sm.stats.multitest.multipletests(p_s, alpha=0.01, method="fdr_bh")
    logq_s=-np.log10(q_s)

    #now add all info to pop data table
    columns = ["chrom", "pos", "h", "z", "p", "q", "-log10(q)"]
    df = pd.DataFrame(data = [chrom_s, pos_s, h_s, z_s[1:], p_s[1:], q_s[1:], logq_s[1:]]).T
    df.columns=columns

    #get cumul positions
    fig_x_pos_s=[]
    for index, row in df.iterrows(): 
        fig_x_pos_s.append(int(row["pos"]) + int(cumul_start[row['chrom']]))

    df['fig_x_pos']=fig_x_pos_s

    #save data to csv file
    csv_file ="results/n10/{}_n10_hscan.csv".format(pop, pop)
    df = df.sort_values(["fig_x_pos"], ascending = True)
    df.to_csv(csv_file, index=False, header=True, mode='w')

    #-------------------------------------------------------------------------------
    # generate figure
   
    #iterate over the df and get the chrom, cumul_pos, and log10pvalue
    fig_chrom_colors =[]

    chrom_colors={}
    chrom_colors["SM_V7_1"] = "black"
    chrom_colors["SM_V7_2"] = "darkgray"
    chrom_colors["SM_V7_3"] = "black"
    chrom_colors["SM_V7_4"] = "darkgray"
    chrom_colors["SM_V7_5"] = "black"
    chrom_colors["SM_V7_6"] = "darkgray"
    chrom_colors["SM_V7_7"] = "black"
    sig_color = "blue"

    for index, row in df.iterrows(): 
        fig_chrom_colors.append(chrom_colors[row['chrom']])
 
    #update colors of significant peaks
    for index, row in df.iterrows(): 
        if row.q<sidak and row.h>h_max:
            fig_chrom_colors[index]=sig_color
    
    #plot log data for H
    ticks = [ (cumul_start['SM_V7_1'] + cumul_start['SM_V7_2'] )/2,
              (cumul_start['SM_V7_2'] + cumul_start['SM_V7_3'] )/2,
              (cumul_start['SM_V7_3'] + cumul_start['SM_V7_4'] )/2,
              (cumul_start['SM_V7_4'] + cumul_start['SM_V7_5'] )/2,
              (cumul_start['SM_V7_5'] + cumul_start['SM_V7_6'] )/2,
              (cumul_start['SM_V7_6'] + cumul_start['SM_V7_7'] )/2,
              (cumul_start['SM_V7_7'] + scanned_size )/2 ]

    tick_lbls = [ "1", "2", "3", "4", "5", "6" ,"7"]

    plt.scatter(df['fig_x_pos'], df['-log10(q)'], marker =".", s = 1, c = fig_chrom_colors) 
    plt.ylabel("-log10(q)")
    #plot FDR line
    plt.axhline(y=np.log10(sidak)*-1, color="black", linestyle=":", linewidth=1)
    #plt.text(df['fig_x_pos'][-1:], np.log10(sidak)*-1, 'FDR=0.01', horizontalalignment='right')
    #plot max_h line)
    plt.axhline(y=logq_s[0], color="black", linestyle="--", linewidth=1)
    #plt.text(df['fig_x_pos'][-1:], logq_s[0], 'max(h) neutral', horizontalalignment='right')
    plt.xticks(ticks, tick_lbls) 
    plt.title("{} H".format(pop))
    plt.savefig("results/hscan/{}_hscan_q.png".format(pop, pop), dpi=300) 
    plt.close()


    plt.scatter(df['fig_x_pos'], df['h'], marker =".", s = 1, c = fig_chrom_colors) 
    plt.ylabel("h")
    #plt.axhline(y=min(h_s[reject_s[1:]]), color="black", linestyle=":", linewidth=1)
    #plt.axhline(y=h_max, color="black", linestyle="--", linewidth=1)
    plt.xticks(ticks, tick_lbls) 
    plt.title("{} H".format(pop))
    plt.savefig("results/n10/{}_n10_hscan_h.png".format(pop, pop), dpi=300)
    plt.close()

niger


NameError: name 'h_max' is not defined

## SWEEPFINDER

In [11]:
%%bash

if [ ! -d "results/n10/sweepfinder" ] 
then
    mkdir -p results/n10/sweepfinder
fi

cd results/n10/sweepfinder

#convert to sweefinder2 format
vcftools \
    --vcf ../smv7_ex_autosomes_n10.vcf \
    --counts2 \
    --out smv7_ex_autosomes_n10_tmp

tail -n+2 smv7_ex_autosomes_n10_tmp.frq.count \
    | awk -v OFS="\t" '{print $2,$6,$4,"1"}' \
    >smv7_ex_autosomes_n10.in

echo -e 'position\tx\tn\tfolded' \
    | cat - smv7_ex_autosomes_n10.in \
    > temp && mv temp smv7_ex_autosomes_n10.in

#calculate genome-wide sfs
SweepFinder2 -f smv7_ex_autosomes_n10.in smv7_ex_autosomes_n10.SpectFile

done readsnps datasize=475081 nmax=80 nmin=24 xmax=81 invar=2
getfreq rdatasize=1327
calling findmax numpar=80
RUNNING THE L-BFGS-B CODE

	* * *

Machine precision=2.220446e-16
N = 80    M = 12
ITERATION 0
At iterate    0 f=449028.786508	|proj g| = 1.732743e+01

	* * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cuachy point
Projg = norm of the final projected gradient
F     = final function value

	* * *
N	Tit	Tnf	Tnint	Skip	Nact	Projg	F
80	15	19	41	0	0	1.222590e+01	4.488109e+05
 F = 448810.933613
CONVERGENCE: REL_REDUCTION_OF_F <= FACTR*EPSMCH            
Cauchy time 0 seconds.
Subspace minimization time 0 seconds.
Line search time 13 seconds.
Total User time 14 seconds.

frequency spectrum written to smv7_ex_autosomes_n10.SpectFile



VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--vcf ../smv7_ex_autosomes_n10.vcf
	--counts2
	--out smv7_ex_autosomes_n10_tmp

After filtering, kept 40 out of 40 Individuals
Outputting Frequency Statistics...
After filtering, kept 475081 out of a possible 475081 Sites
Run Time = 11.00 seconds


### run sweepfinder on the actual data

code/sweepfinder2-run_with_real_data.sh

In [20]:
%%bash

cd results/n10/sweepfinder


for POP in brazil senegal niger tanzania; do
    mkdir $POP
    
    #get the pop specific sfs
    vcftools \
        --keep ../"$POP"_n10.list \
        --vcf ../smv7_ex_autosomes_n10.vcf  \
        --counts2 \
        --stdout \
        >$POP/$POP.freq

    tail -n+2 $POP/$POP.freq  | awk -v OFS="\t" '{print $2,$6,$4,"1"}' >$POP/$POP.in

    echo -e 'position\tx\tn\tfolded' | cat - $POP/$POP.in > temp && mv temp $POP/$POP.in

    SweepFinder2 -f $POP/$POP.in $POP/$POP.sfs
 
    #run sweepfinder for each chrom
    for I in $(seq 1 7); do
        CHR=SM_V7_$I

        #convert vcf to sw input format
        vcftools \
            --keep ../"$POP"_n10.list \
            --vcf ../smv7_ex_autosomes_n10.vcf \
            --chr $CHR \
            --counts2 \
            --stdout \
            >$POP/$CHR"_"$POP.freq

        tail -n+2 $POP/$CHR"_"$POP.freq  | awk -v OFS="\t" '{print $2,$6,$4,"1"}' >$POP/$CHR"_"$POP.in

        echo -e 'position\tx\tn\tfolded' | cat - $POP/$CHR"_"$POP.in > temp && mv temp $POP/$CHR"_"$POP.in

        #submit sweepfinder        
        CMD="conda activate sch_man_nwinvasion-sweepfinder; SweepFinder2 -lg 1000 $POP/$CHR"_"$POP.in $POP/$POP.sfs $POP/$CHR"_"$POP.sw2out"

        echo $CMD | qsub -V -cwd -S /bin/bash -q all.q -j y -N sf2r"$I"_"$POP" -o $POP/real_$CHR.log -pe smp 2
    done
done

Your job 5666836 ("sf2r1_brazil") has been submitted
Your job 5666837 ("sf2r2_brazil") has been submitted
Your job 5666838 ("sf2r3_brazil") has been submitted
Your job 5666839 ("sf2r4_brazil") has been submitted
Your job 5666840 ("sf2r5_brazil") has been submitted
Your job 5666841 ("sf2r6_brazil") has been submitted
Your job 5666842 ("sf2r7_brazil") has been submitted
Your job 5666843 ("sf2r1_senegal") has been submitted
Your job 5666844 ("sf2r2_senegal") has been submitted
Your job 5666845 ("sf2r3_senegal") has been submitted
Your job 5666846 ("sf2r4_senegal") has been submitted
Your job 5666847 ("sf2r5_senegal") has been submitted
Your job 5666848 ("sf2r6_senegal") has been submitted
Your job 5666849 ("sf2r7_senegal") has been submitted
Your job 5666850 ("sf2r1_niger") has been submitted
Your job 5666851 ("sf2r2_niger") has been submitted
Your job 5666852 ("sf2r3_niger") has been submitted
Your job 5666853 ("sf2r4_niger") has been submitted
Your job 5666854 ("sf2r5_niger") has been s

mkdir: cannot create directory `brazil': File exists

VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--vcf ../smv7_ex_autosomes_n10.vcf
	--chr SM_V7_1
	--keep ../brazil_n10.list
	--counts2
	--stdout

Keeping individuals in 'keep' list
After filtering, kept 10 out of 40 Individuals
Outputting Frequency Statistics...
After filtering, kept 162219 out of a possible 475081 Sites
Run Time = 6.00 seconds

VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--vcf ../smv7_ex_autosomes_n10.vcf
	--chr SM_V7_2
	--keep ../brazil_n10.list
	--counts2
	--stdout

Keeping individuals in 'keep' list
After filtering, kept 10 out of 40 Individuals
Outputting Frequency Statistics...
After filtering, kept 78902 out of a possible 475081 Sites
Run Time = 4.00 seconds

VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--vcf ../smv7_ex_autosomes_n10.vcf
	--chr SM_V7_3
	--keep ../brazil_n10.list
	

### run sweepfinder on simulated data
code/sweepfinder2-run_with_simulated_data.sh

In [ ]:
%%bash

cd results/n10/sweepfinder

CONDA="conda activate sch_man_nwinvasion-sweepfinder"
QSUB="qsub -V -cwd -S /bin/bash -q all.q -j y -pe smp 2 "    

mkdir logs

for POP in senegal tanzania niger brazil; do
    mkdir $POP-sim

    for PROBED_VCF in $(ls ~/sch_man_nwinvasion/results/n10/msprime/$POP/chr1_"$POP"_rep_*seed_*_probed.vcf); do
        SAMPLE_NAME=$(basename $PROBED_VCF .vcf)
        
        #convert to sweep format
        vcftools \
            --vcf $PROBED_VCF  \
            --counts2 \
            --stdout \
            >$POP-sim/$SAMPLE_NAME.freq

        tail -n+2 $POP-sim/$SAMPLE_NAME.freq  | awk -v OFS="\t" '{print $2,$6,$4,"1"}' >$POP-sim/$SAMPLE_NAME.in

        echo -e 'position\tx\tn\tfolded' | cat - $POP-sim/$SAMPLE_NAME.in > temp && mv temp $POP-sim/$SAMPLE_NAME.in

        #generate sfs
        SFS_CMD="SweepFinder2 -f $POP-sim/$SAMPLE_NAME.in $POP-sim/$SAMPLE_NAME.sfs >logs/"$SAMPLE_NAME"_sfs.log"

        #gen sweepfinder2
        SF2_CMD="SweepFinder2 -lg 1000 $POP-sim/$SAMPLE_NAME.in $POP-sim/$SAMPLE_NAME.sfs $POP-sim/$SAMPLE_NAME.sw2out"
        
        #submit to the queue
        CMD="$CONDA; $SFS_CMD; $SF2_CMD"
        echo $CMD | $QSUB -N sf2s_$SAMPLE_NAME -o logs/sf2_sim_$SAMPLE_NAME.log

    done
done


## FST

In [25]:
import os
import shutil
import allel
import math
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
from scipy import stats
from scipy import signal

os.chdir("/master/nplatt/sch_man_nwinvasion")
os.mkdir("results/n10/fst")

with open('data/pop_assign.yml') as yaml_file:
    pop_assign = yaml.load(yaml_file, Loader=yaml.FullLoader)

#-----------------------------------
# get lengths from cumul positions
#make sure that all stops are not gt chrom length
chr_length = {}
#genome_size = 0
with open('/master/nplatt/sch_man_nwinvasion/data/genomes/Smansoni_v7.fa.fai', 'r') as fai:
    for entry in fai:
        chrom, length, *offset = entry.rstrip().split("\t")
        chr_length[chrom]=int(length)

    cumul_start={}
    cumul_start['SM_V7_1']=0
    cumul_start['SM_V7_2']= cumul_start['SM_V7_1'] + chr_length['SM_V7_1']
    cumul_start['SM_V7_3']= cumul_start['SM_V7_2'] + chr_length['SM_V7_2']
    cumul_start['SM_V7_4']= cumul_start['SM_V7_3'] + chr_length['SM_V7_3']
    cumul_start['SM_V7_5']= cumul_start['SM_V7_4'] + chr_length['SM_V7_4']
    cumul_start['SM_V7_6']= cumul_start['SM_V7_5'] + chr_length['SM_V7_5']
    cumul_start['SM_V7_7']= cumul_start['SM_V7_6'] + chr_length['SM_V7_6']
    scanned_size = cumul_start['SM_V7_7'] + chr_length['SM_V7_7']

#-------------------------------------------------------------------------------
# get genotype info per population

#read in vcf
callset=allel.read_vcf('results/n10/smv7_ex_autosomes_n10.vcf')

#now get an index for each sample/population
samples = callset["samples"]

i=0 
pop_idxs = defaultdict(list)   
for sample in samples:  
     pop_idxs[pop_assign[sample]].append(i) 
     i=i+1 

pops= list(pop_idxs.keys()) 

#get genotypes
gt=allel.GenotypeArray(callset['calldata/GT'])

#now get allele count per population
ac=gt.count_alleles()

#for simplicity add maf info to callset data
maf=ac[:, :2].min(axis=1)/ac[:, :2].sum(axis=1)
callset['maf']=maf 

pop_ac={}
for pop in pops:
    pop_ac[pop] = gt.count_alleles(subpop=pop_idxs[pop])
    
#-------------------------------------------------------------------------------
#generate windows
window=100_000

#define an array of window start and stops
window_starts = [int(x - (window/2)) for x in callset['variants/POS']]
window_stops  = [int(x + (window/2)) for x in callset['variants/POS']]

#make sure that window starts are all gt 1
window_starts = [1 if i < 1 else i for i in window_starts]


#make sure that all stops are not gt chrom length
chr_length = {}
#genome_size = 0
with open('/master/nplatt/sch_man_nwinvasion/data/genomes/Smansoni_v7.fa.fai', 'r') as fai:
    for entry in fai:
        chrom, length, *offset = entry.rstrip().split("\t")
        chr_length[chrom]=int(length)
        #genome_size = genome_size + chr_length[chrom]
    
i=0
for stop in window_stops:
    chrom = callset['variants/CHROM'][i]
    
    if stop > chr_length[chrom]:
        window_stops[i]=chr_length[chrom]
    i=i+1
    
windows = np.column_stack((np.array(window_starts), 
                           np.array(window_stops)))

callset['windows']=windows

In [ ]:
#-------------------------------------------------------------------------------
# fst calculations

pops = ["brazil", "tanzania", "niger", "senegal" ]

idx_comps = {"brazil":   [pop_idxs["brazil"],   pop_idxs["tanzania"] + pop_idxs["niger"]  + pop_idxs["senegal"] ],
             "tanzania": [pop_idxs["tanzania"], pop_idxs["brazil"]   + pop_idxs["niger"]  + pop_idxs["senegal"] ],
             "niger":    [pop_idxs["niger"],    pop_idxs["tanzania"] + pop_idxs["brazil"] + pop_idxs["senegal"] ],
             "senegal":  [pop_idxs["senegal"],  pop_idxs["tanzania"] + pop_idxs["niger"]  + pop_idxs["brazil"] ]}

#make comparisons between population
for pop in idx_comps.keys():
    pop1_idx = idx_comps[pop][0]
    pop2_idx = idx_comps[pop][1]

    fst_s             = []
    fst_calc_window_s = []
    fst_count_s       = []

    #create empty dataframe to store data    
    headers = ["chrom", "pos", "fst", "smoothed_fst", "window", "num_snps", "zscore", "pvalue", "-log10(p)"]
    df=pd.DataFrame(columns=headers) 

    #now loop through each chromosome
    for chrom in list(set(callset['variants/CHROM'])) :
        target_sites = np.logical_and( callset['maf'] < 0.05, 
                                       callset['variants/CHROM'] == chrom )  

        chr_gts  = gt[target_sites]
        chr_poss = callset['variants/POS'][target_sites]
        chr_wins = callset['windows'][target_sites]

        
        fsts, fst_calc_windows, fst_counts =allel.windowed_weir_cockerham_fst(chr_poss, chr_gts, subpops=[pop1_idx, pop2_idx], windows=chr_wins )

        #get rid of nan values
        useful_values = np.logical_and( np.isfinite(fsts), fst_counts>=10) 

        fsts = fsts[useful_values]
        fst_calc_windows = fst_calc_windows[useful_values]
        fst_counts = fst_counts[useful_values]
        chr_poss = chr_poss[useful_values]

        #set negative fst values to 0
        i=0
        for fst in fsts:
            if fst <0:
                fsts[i]=0
            i=i+1        
        
        #smooth
        smoothed_fsts=signal.medfilt(fsts, kernel_size = 101)

        #calculate pvalue
        zs = stats.zscore(fsts)
        ps = stats.norm.cdf(zs)
        logps = [-1*np.log10(p) for p in ps] 

        #add data to dataframe/table
        data = list(zip([chrom]*len(fsts), chr_poss, fsts, smoothed_fsts, fst_calc_windows, fst_counts, zs, ps, logps))
        chr_df=pd.DataFrame(data, columns=headers)
        df = df.append(chr_df)

    #add cumul positions
    fig_x_pos_s=[]
    for index, row in df.iterrows(): 
        fig_x_pos_s.append(int(row["pos"]) + int(cumul_start[row['chrom']]))

    df['fig_x_pos']=fig_x_pos_s

    #save data to csv file
    csv_file = "./results/n10/fst/{}_vs_all_windowed_fst.csv".format(pop)
    df = df.sort_values(["fig_x_pos"], ascending = True)
    df.to_csv(csv_file, index=False, header=True, mode='w')

/master/nplatt/miniconda3/envs/sch_man_nwinvasion-dcms/lib/python3.8/site-packages/allel/stats/fst.py:187: RuntimeWarning: invalid value encountered in true_divide
  p = ac / an[:, np.newaxis, :]
/master/nplatt/miniconda3/envs/sch_man_nwinvasion-dcms/lib/python3.8/site-packages/allel/stats/fst.py:228: RuntimeWarning: divide by zero encountered in true_divide
  a = ((n_bar / n_C) *
/master/nplatt/miniconda3/envs/sch_man_nwinvasion-dcms/lib/python3.8/site-packages/allel/stats/fst.py:412: RuntimeWarning: invalid value encountered in double_scalars
  return np.nansum(wa) / (np.nansum(wa) + np.nansum(wb) + np.nansum(wc))


In [6]:
#-------------------------------------------------------------------------------
# generate figure
   
for pop in pops:
    csv_file = "results/n10/{}n10_vs_all_windowed_fst.csv".format(pop)
    df = pd.read_csv(csv_file,sep=",")

    #remove sites lt 10 snps?

    #get cumul positions
    cumul_start={}
    cumul_start['SM_V7_1']=0
    cumul_start['SM_V7_2']= cumul_start['SM_V7_1'] + chr_length['SM_V7_1']
    cumul_start['SM_V7_3']= cumul_start['SM_V7_2'] + chr_length['SM_V7_2']
    cumul_start['SM_V7_4']= cumul_start['SM_V7_3'] + chr_length['SM_V7_3']
    cumul_start['SM_V7_5']= cumul_start['SM_V7_4'] + chr_length['SM_V7_4']
    cumul_start['SM_V7_6']= cumul_start['SM_V7_5'] + chr_length['SM_V7_5']
    cumul_start['SM_V7_7']= cumul_start['SM_V7_6'] + chr_length['SM_V7_6']
    scanned_size = cumul_start['SM_V7_7'] + chr_length['SM_V7_7']

    #iterate over the df and get the chrom, cumul_pos, and log10pvalue
    fig_x_pos=[]
    fig_chrom_colors =[]

    chrom_colors={}
    chrom_colors["SM_V7_1"] = "black"
    chrom_colors["SM_V7_2"] = "darkgray"
    chrom_colors["SM_V7_3"] = "black"
    chrom_colors["SM_V7_4"] = "darkgray"
    chrom_colors["SM_V7_5"] = "black"
    chrom_colors["SM_V7_6"] = "darkgray"
    chrom_colors["SM_V7_7"] = "black"
    sig_color = "blue"

    for index, row in df.iterrows(): 
        fig_x_pos.append(row["pos"] + cumul_start[row['chrom']])
        fig_chrom_colors.append(chrom_colors[row['chrom']])
 
    
    ticks = [ (cumul_start['SM_V7_1'] + cumul_start['SM_V7_2'] )/2,
              (cumul_start['SM_V7_2'] + cumul_start['SM_V7_3'] )/2,
              (cumul_start['SM_V7_3'] + cumul_start['SM_V7_4'] )/2,
              (cumul_start['SM_V7_4'] + cumul_start['SM_V7_5'] )/2,
              (cumul_start['SM_V7_5'] + cumul_start['SM_V7_6'] )/2,
              (cumul_start['SM_V7_6'] + cumul_start['SM_V7_7'] )/2,
              (cumul_start['SM_V7_7'] + scanned_size )/2 ]

    tick_lbls = [ "1", "2", "3", "4", "5", "6" ,"7"]

    plt.scatter(fig_x_pos, df['smoothed_fst'], marker =".", s = 1, c = fig_chrom_colors) 
    plt.ylabel("smoothed_fst") 
    plt.xticks(ticks, tick_lbls) 
    plt.title("Fst ({})".format(pop))
    plt.savefig('results/n10/{}n10_vs_all_windowed_fst.png'.format(pop), dpi=300) 
    plt.close()

## Prep DCMS

## DCMS